#  迁移学习

In [ ]:
import os
import shutil
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications import *
import matplotlib.pyplot as plt


In [4]:
#参数

train_file_name='train'
valid_file_name='valid'
IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
batch_size=32

nb_epoch =3
FC_SIZE = 1024

In [1]:
#图片分类，创建软链接
train_filenames = os.listdir('imagedata/train')
valid_count=int(len(train_filenames)*0.1)

valid_data=train_filenames[-valid_count:]
train_data=train_filenames[:-valid_count]

train_cat = filter(lambda x:x[:3] == 'cat', train_data)
train_dog = filter(lambda x:x[:3] == 'dog', train_data)

valid_cat = filter(lambda x:x[:3] == 'cat', valid_data)
valid_dog = filter(lambda x:x[:3] == 'dog', valid_data)

def cre_rem_dir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cre_rem_dir(train_file_name)
os.mkdir(train_file_name+'/cat')
os.mkdir(valid_file_name+'/dog')

cre_rem_dir(valid_file_name)
os.mkdir(valid_file_name+'/cat')
os.mkdir(valid_file_name+'/dog')


cur_path=os.getcwd()

for filename in train_cat:
    os.symlink(cur_path+'/imagedata/train/'+filename, train_file_name+'/cat/'+filename)
for filename in train_dog:
    os.symlink(cur_path+'/imagedata/train/'+filename, train_file_name+'/dog/'+filename)
for filename in valid_cat:
    os.symlink(cur_path+'/imagedata/train/'+filename, valid_file_name+'/cat/'+filename)
for filename in valid_dog:
    os.symlink(cur_path+'/imagedata/train/'+filename, valid_file_name+'/dog/'+filename)    

In [2]:
#数据增强
train_datagen =  ImageDataGenerator(
    preprocessing_function=inception_v3.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=inception_v3.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
  'train',
  target_size=(IM_WIDTH, IM_HEIGHT),
  batch_size=batch_size,
)
validation_generator = test_datagen.flow_from_directory(
  'valid',
  target_size=(IM_WIDTH, IM_HEIGHT),
  batch_size=batch_size,
)

Using TensorFlow backend.


Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [5]:
base_model = InceptionV3(weights='imagenet', include_top=False)

#添加新层
def add_new_last_layer(base_model, nb_classes):
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x) 
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(input=base_model.input, output=predictions)
    return model


def setup_to_transfer_learn(model, base_model):
    for layer in base_model.layers:
        layer.trainable = False
        model.compile(optimizer='rmsprop',
                      loss='categorical_crossentropy', 
                      metrics=['accuracy'])

        
model=add_new_last_layer(base_model,2)

setup_to_transfer_learn(model,base_model)

/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
history = model.fit_generator(
  train_generator,
  nb_epoch=nb_epoch,
  validation_data=validation_generator,
  class_weight='auto')

model.save(args.output_model_file)

/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/Users/hl/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., class_weight="auto", epochs=5)`
  """


Epoch 1/5
1407/1407 [==============================] - 13450s 10s/step - loss: 0.2849 - acc: 0.9080 - val_loss: 0.0798 - val_acc: 0.9668
Epoch 2/5
 239/1407 [====>.........................] - ETA: 67:53:34 - loss: 0.1469 - acc: 0.9383

In [5]:
#可视化

def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
  
    plt.show()

plot_training(history)    

NameError: name 'history' is not defined